In [8]:
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

# add the path to my packages to system paths so they can be imported
import sys
sys.path.append('/home/yasamanparhizkar/Documents/yorku/01_thesis/code/my_packages')
# sys.path.append('F:\MAScThesis\code\my_packages')
# sys.path.append('/home/yasamanparhizkar/Documents/thesis/code/my_packages')

import dataprocess.data_handler_01 as dh
warnings.filterwarnings("ignore")

In [9]:
diamonds = sns.load_dataset("diamonds")
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [10]:
type(diamonds)

pandas.core.frame.DataFrame

In [11]:
print(diamonds)

       carat        cut color clarity  depth  table  price     x     y     z
0       0.23      Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1       0.21    Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2       0.23       Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3       0.29    Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4       0.31       Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75
...      ...        ...   ...     ...    ...    ...    ...   ...   ...   ...
53935   0.72      Ideal     D     SI1   60.8   57.0   2757  5.75  5.76  3.50
53936   0.72       Good     D     SI1   63.1   55.0   2757  5.69  5.75  3.61
53937   0.70  Very Good     D     SI1   62.8   60.0   2757  5.66  5.68  3.56
53938   0.86    Premium     H     SI2   61.0   58.0   2757  6.15  6.12  3.74
53939   0.75      Ideal     D     SI2   62.2   55.0   2757  5.83  5.87  3.64

[53940 rows x 10 columns]


# Create a pandas dataframe from fish movie features

In [12]:
# load all spike data from file
spikes_dp = '../../data/original_files/spikes.csv'
binned_data = np.loadtxt(spikes_dp, delimiter=',')
binned_data = binned_data.reshape(binned_data.shape[0], 1141, 113)
binned_data = binned_data * 2 - 1     # turn labels from 0,1 to -1,1

I_order_10 = [54, 35, 10, 60, 74, 9, 61, 56, 91, 104]

# group all neurons together
grouped_data = np.zeros((297, 1141, 1))
for trial in range(297):
    for frame in range(1141):
        grouped_data[trial, frame, :] = 2 * int((binned_data[trial, frame, :] == 1).any()) - 1
        
print('grouped_data.shape = ', grouped_data.shape)

grouped_data.shape =  (297, 1141, 1)


In [13]:
# use only the second trial
ind_min = 1*1141+0
ind_max = 2*1141-1

def transform(fv):
    """
    Transform to be applied on feature vectors.
    
    Input: fv
    fv - 1xDf torch tensor representing a feature vector
    
    Output: fvv
    fvv - 1xDf' torch tensor representing the transformed feature vector
    """
    
    # for faster run and less memory usage
    fvv = fv[:, ::10]
    
    # for numerical stability during GD
    # fvv = fvv * 10
    
    return fvv

data_params = {'func': dh.datapoint_torch, 'features_dp': '../../data/features/slowfast/slowfast_4732/', \
               'spike_data': grouped_data, 'group_id': 0, 'transform': transform}

train_num = 500
val_num = 100
seed = None

# create training and validation datasets
train_num, val_num, train_data, val_data = dh.random_train_val(train_num, val_num, ind_min, ind_max, data_params, seed)

print('# training datapoints: ', train_num)
print('# validation datapoints: ', val_num)
print('features array shape (NxDf): ', train_data['des'].shape)
print('a few rows of the training feature array:\n',train_data['des'][:5])

# training datapoints:  500
# validation datapoints:  100
features array shape (NxDf):  (500, 474)
a few rows of the training feature array:
 [[2.7591696 1.8667322 1.996728  ... 2.6279461 5.0802674 2.6255097]
 [2.8032546 1.912655  1.958082  ... 2.5674512 5.029712  1.931703 ]
 [2.7883947 1.8380947 1.9161978 ... 2.6664696 5.081702  1.683916 ]
 [2.779543  1.9785925 2.0078094 ... 2.5862923 5.0203495 2.3367867]
 [2.847968  1.8654196 1.8850592 ... 2.652353  5.129178  1.7659343]]


In [14]:
# create a dataframe from the features array
train_df = pd.DataFrame(data = train_data['des'])
val_df = pd.DataFrame(data = val_data['des'])
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,464,465,466,467,468,469,470,471,472,473
0,2.759170,1.866732,1.996728,2.976990,2.810931,7.295244,4.145206,5.453339,5.457242,2.980649,...,2.136682,2.749541,2.157637,4.776975,2.166260,3.885952,1.991067,2.627946,5.080267,2.625510
1,2.803255,1.912655,1.958082,3.214768,2.767216,7.468374,6.002043,5.394676,5.492955,3.047853,...,2.192616,2.740751,1.874332,4.670727,2.187642,2.397994,2.014503,2.567451,5.029712,1.931703
2,2.788395,1.838095,1.916198,3.102780,2.819376,7.397983,4.087776,5.496922,5.490079,3.053795,...,2.114702,2.774444,2.119722,4.717732,2.190404,2.393993,2.017659,2.666470,5.081702,1.683916
3,2.779543,1.978593,2.007809,3.218792,2.819501,7.348711,4.212461,5.478083,5.452457,3.076757,...,1.779368,2.723620,2.068911,4.739473,2.220844,2.282309,3.004078,2.586292,5.020350,2.336787
4,2.847968,1.865420,1.885059,3.113307,2.719427,7.406642,4.111432,5.532706,5.426259,3.081644,...,2.159604,2.756416,2.048295,4.684746,2.196888,2.370861,1.994968,2.652353,5.129178,1.765934


In [15]:
# create pandas series from the labels
train_y = pd.DataFrame(data = train_data['lbls'])
val_y = pd.DataFrame(data = val_data['lbls'])
train_y.head()

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,-1.0


In [16]:
# create xgb Dmatrices from dataframes
dtrain_from_df = xgb.DMatrix(train_df, train_y)
print('Dmatrix shape: {} x {}'.format(dtrain_from_df.num_row(), dtrain_from_df.num_col()))
print('Dmatrix labels: ', dtrain_from_df.get_label())

Dmatrix shape: 500 x 474
Dmatrix labels:  [ 1.  1.  1.  1. -1. -1. -1. -1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1.
  1.  1.  1.  1. -1. -1.  1. -1. -1.  1.  1.  1. -1. -1. -1.  1.  1. -1.
  1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1. -1.  1.
  1.  1.  1.  1.  1. -1. -1.  1. -1. -1.  1.  1.  1.  1. -1.  1.  1.  1.
  1. -1. -1. -1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.
 -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.  1.
  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1. -1.  1.  1.
  1.  1.  1.  1. -1.  1. -1.  1.  1. -1.  1. -1.  1.  1. -1.  1.  1.  1.
  1. -1. -1.  1.  1. -1.  1. -1. -1. -1.  1.  1.  1.  1.  1.  1. -1.  1.
  1.  1.  1. -1.  1.  1.  1.  1. -1.  1.  1. -1.  1.  1. -1.  1.  1.  1.
 -1.  1. -1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1.  1. -1.  1.  1.
  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1. -1.  1. -1. -1.  1. -1.
 -1. -1. -1.  1.  1.  1. -1. -1.  1.  1. -1.  1. -1.  1.  1. -1. -1.  1.
 -1. -1. 

In [17]:
# create xgb Dmatrices directly from numpy arrays
dtrain = xgb.DMatrix(train_data['des'], label=(train_data['lbls']+1)//2)
dval = xgb.DMatrix(val_data['des'], label=(val_data['lbls']+1)//2)

In [18]:
print('Train Dmatrix shape: {} x {}'.format(dtrain.num_row(), dtrain.num_col()))
print('Val Dmatrix shape: {} x {}'.format(dval.num_row(), dval.num_col()))
print('Train Dmatrix labels: ', dtrain.get_label())
print('Val Dmatrix labels: ', dval.get_label())

Train Dmatrix shape: 500 x 474
Val Dmatrix shape: 100 x 474
Train Dmatrix labels:  [1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0.
 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1.
 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0.
 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1.
 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0

# Training

In [19]:
# Define hyperparameters
params = {"objective": "binary:logistic", "tree_method": "hist"}
evals = [(dval, "validation"), (dtrain, "train")]
n = 20
veval = 1

model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=n,
    evals=evals,
    verbose_eval=veval # print losses every ten rounds
)

[0]	validation-logloss:0.58452	train-logloss:0.50568
[1]	validation-logloss:0.54146	train-logloss:0.39630
[2]	validation-logloss:0.53360	train-logloss:0.31201
[3]	validation-logloss:0.47785	train-logloss:0.26020
[4]	validation-logloss:0.44765	train-logloss:0.20970
[5]	validation-logloss:0.45535	train-logloss:0.17382
[6]	validation-logloss:0.44714	train-logloss:0.14464
[7]	validation-logloss:0.45386	train-logloss:0.12570
[8]	validation-logloss:0.43881	train-logloss:0.10713
[9]	validation-logloss:0.43585	train-logloss:0.09323
[10]	validation-logloss:0.44369	train-logloss:0.08163
[11]	validation-logloss:0.42999	train-logloss:0.07193
[12]	validation-logloss:0.44436	train-logloss:0.06444
[13]	validation-logloss:0.44129	train-logloss:0.05770
[14]	validation-logloss:0.44271	train-logloss:0.05195
[15]	validation-logloss:0.44218	train-logloss:0.04711
[16]	validation-logloss:0.44021	train-logloss:0.04310
[17]	validation-logloss:0.43263	train-logloss:0.04019
[18]	validation-logloss:0.43336	train-

In [20]:
model2 = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=n,
    evals=evals,
    verbose_eval=veval, # print losses every ten rounds
    early_stopping_rounds=1 # activate early stopping
)

[0]	validation-logloss:0.58452	train-logloss:0.50568
[1]	validation-logloss:0.54146	train-logloss:0.39630
[2]	validation-logloss:0.53360	train-logloss:0.31201
[3]	validation-logloss:0.47785	train-logloss:0.26020
[4]	validation-logloss:0.44765	train-logloss:0.20970
[5]	validation-logloss:0.45535	train-logloss:0.17382
[6]	validation-logloss:0.44714	train-logloss:0.14464
[7]	validation-logloss:0.45386	train-logloss:0.12570
[8]	validation-logloss:0.43881	train-logloss:0.10713
[9]	validation-logloss:0.43585	train-logloss:0.09323
[10]	validation-logloss:0.44369	train-logloss:0.08163
[11]	validation-logloss:0.42999	train-logloss:0.07193
[12]	validation-logloss:0.44436	train-logloss:0.06444
[13]	validation-logloss:0.44129	train-logloss:0.05770
[14]	validation-logloss:0.44271	train-logloss:0.05195
[15]	validation-logloss:0.44218	train-logloss:0.04711
[16]	validation-logloss:0.44021	train-logloss:0.04310
[17]	validation-logloss:0.43263	train-logloss:0.04019
[18]	validation-logloss:0.43336	train-

In [21]:
# cross-validation
results = xgb.cv(
    params=params, 
    dtrain=dtrain,
    num_boost_round=n,
    nfold=5,
    metrics=["logloss", "auc", "error"]
)

In [22]:
results.head()

,train-logloss-mean,train-logloss-std,train-auc-mean,train-auc-std,train-error-mean,train-error-std,test-logloss-mean,test-logloss-std,test-auc-mean,test-auc-std,test-error-mean,test-error-std
0,0.517488,0.007037,0.958555,0.008912,0.0855,0.013910,0.597818,0.018402,0.770254,0.043862,0.270,0.052154
1,0.402013,0.006084,0.992731,0.002673,0.0345,0.002915,0.557608,0.016179,0.776070,0.026771,0.250,0.035777
2,0.315603,0.006837,0.998921,0.000504,0.0175,0.005244,0.524179,0.029972,0.791803,0.042354,0.230,0.048580
3,0.250724,0.005568,0.999871,0.000162,0.0070,0.004301,0.512341,0.045402,0.797235,0.046004,0.256,0.049234
4,0.203055,0.006131,0.999994,0.000012,0.0020,0.001871,0.498204,0.045781,0.802969,0.044859,0.242,0.038678


In [23]:
# hyperparameter tuning with cross-validation

# Validation

In [24]:
preds = model.predict(dval)

In [25]:
print('preds tyep: ', type(preds))
print('preds shape: ', preds.shape)
print('preds:\n',preds)

preds tyep:  <class 'numpy.ndarray'>
preds shape:  (100,)
preds:
 [0.47663748 0.94244725 0.98170704 0.7972087  0.9287665  0.960873
 0.9545406  0.35761297 0.49313495 0.9769782  0.9952113  0.9840507
 0.14112465 0.14745219 0.9381952  0.99269146 0.9527813  0.83531755
 0.9938188  0.49600095 0.04484522 0.05221152 0.97403073 0.9522482
 0.34073365 0.9891644  0.99474394 0.9664982  0.22732837 0.8417644
 0.52974635 0.8020295  0.9766158  0.41180912 0.08316467 0.40646678
 0.9560419  0.6481282  0.9558457  0.5314252  0.18986082 0.05385704
 0.6073898  0.9349979  0.9644912  0.74844396 0.8818388  0.9886568
 0.9659773  0.8421823  0.9158364  0.9862714  0.9880265  0.5920399
 0.94447607 0.7483455  0.26013133 0.7000498  0.98514986 0.06564195
 0.34448907 0.9704428  0.284082   0.47689733 0.8079233  0.77404004
 0.8858664  0.8267679  0.9794819  0.9759866  0.98327786 0.99400043
 0.98869586 0.78295493 0.98811173 0.03939851 0.8620954  0.22117925
 0.9417132  0.8161011  0.9311473  0.9804186  0.93364334 0.9813083
 0.9

In [31]:
preds_th = (preds > 0.5).astype(int)
gt_lbls = (val_data['lbls']+1)//2
acc = np.sum(preds_th == gt_lbls)/len(gt_lbls)
err = np.sum(preds_th != gt_lbls)/len(gt_lbls)


print('Thresholded preds:\n', preds_th)
print('Ground-truth val. labels:\n', gt_lbls)
print('Val. accuracy: ', acc)
print('Val. error rate: ', err)

Thresholded preds:
 [0 1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 0 0 0 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1
 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1]
Ground-truth val. labels:
 [1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1.
 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1.
 1. 1. 0. 1.]
Val. accuracy:  0.78
Val. error rate:  0.22
